***GENERATED CODE FOR meanshiftmodel PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'user_id'}], 'feature': 'user_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '240', 'mean': '', 'stddev': '', 'min': 'user1', 'max': 'user9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'user_id'}, {'transformation_label': 'String Indexer', 'feature_label': 'user_id'})
        transformationDF = transformationDF.drop('user_id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'hotel_name'}], 'feature': 'hotel_name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '240', 'mean': '', 'stddev': '', 'min': '128 Bangkok', 'max': 'ZEN Rooms Boss Prakanong', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'hotel_name'}, {'transformation_label': 'String Indexer', 'feature_label': 'hotel_name'})
        transformationDF = transformationDF.drop('hotel_name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'trip_type'}], 'feature': 'trip_type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '240', 'mean': '', 'stddev': '', 'min': 'business', 'max': 'leisure', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'trip_type'}, {'transformation_label': 'String Indexer', 'feature_label': 'trip_type'})
        transformationDF = transformationDF.drop('trip_type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'companion'}], 'feature': 'companion', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '240', 'mean': '', 'stddev': '', 'min': 'couple', 'max': 'solo', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'companion'}, {'transformation_label': 'String Indexer', 'feature_label': 'companion'})
        transformationDF = transformationDF.drop('companion')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'destination'}], 'feature': 'destination', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '240', 'mean': '', 'stddev': '', 'min': 'bangkok', 'max': 'bangkok', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'destination'}, {'transformation_label': 'String Indexer', 'feature_label': 'destination'})
        transformationDF = transformationDF.drop('destination')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        stageAttributes['TrainingPercent']


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run meanshiftmodelHooks.ipynb
try:
	#sourcePreExecutionHook()

	hotelrecommend = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/Data/TrainData/recommender/Hotel_Recommend.csv', 'filename': 'Hotel_Recommend.csv', 'token': '', 'delimiter': ','}")
	#sourcePostExecutionHook(hotelrecommend)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run meanshiftmodelHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(hotelrecommend,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "user_id"}], "feature": "user_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "240", "mean": "", "stddev": "", "min": "user1", "max": "user9", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "user_id"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "hotel_name"}], "feature": "hotel_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "240", "mean": "", "stddev": "", "min": "128 Bangkok", "max": "ZEN Rooms Boss Prakanong", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "hotel_name"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rating", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "240", "mean": "8.62", "stddev": "0.51", "min": "7.0", "max": "9.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "rating"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "price", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "240", "mean": "2924.32", "stddev": "2238.03", "min": "230", "max": "13418", "missing": "0"}, "updatedLabel": "price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "near_station", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "240", "mean": "0.19", "stddev": "0.08", "min": "0.0", "max": "0.390243902", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "near_station"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "restaurant", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "240", "mean": "0.36", "stddev": "0.14", "min": "0.2", "max": "0.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "restaurant"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "entertain", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "240", "mean": "0.12", "stddev": "0.08", "min": "0.0", "max": "0.289855072", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "entertain"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "shopping_mall", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "240", "mean": "0.13", "stddev": "0.08", "min": "0.0", "max": "0.350877193", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "shopping_mall"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "convenience_store", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "240", "mean": "0.2", "stddev": "0.07", "min": "0.0", "max": "0.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "convenience_store"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "trip_type"}], "feature": "trip_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "240", "mean": "", "stddev": "", "min": "business", "max": "leisure", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "trip_type"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "companion"}], "feature": "companion", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "240", "mean": "", "stddev": "", "min": "couple", "max": "solo", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "companion"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "destination"}], "feature": "destination", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "240", "mean": "", "stddev": "", "min": "bangkok", "max": "bangkok", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "destination"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run meanshiftmodelHooks.ipynb
try:
	#mlPreExecutionHook()

	clustering = Clustering.run(manualfe,spark,json.dumps( {"autoClustering": 0, "originalfile": "/FileStore/platform/Data/TrainData/recommender/Hotel_Recommend.csv", "distinct_column": "rating", "model": "meanshift", "TrainingPercent": 100, "totalClusters": null, "run_id": "6d7c3356a8064a5f8d444feb17cce456", "model_id": "5f69e217d88384088e1d767d"}))

	#mlPostExecutionHook(clustering)

except Exception as ex: 
	logging.error(ex)
